### Imports

In [26]:
import os
import cv2
import pandas as pd

In [27]:
imgpath = './../../data/ocr/docbank/images/'
txtpath = './../../data/ocr/docbank/ori_txt/'
outpath = './../../data/ocr/docbank/txt/'

### Resize bboxes according to images

In [28]:
labels = set()
for file in os.listdir(txtpath):
    df = pd.read_csv(txtpath + file, delimiter='\t',
                     names=["token", "X1", "Y1", "X2", "Y2", "R", "G", "B", "font name", "label"])
    img = cv2.imread(imgpath + file[:-4] + ".jpg")
    height, width, _ = img.shape
    df['X1'] = df['X1'].apply(lambda x : int(x * width / 1000))
    df['Y1'] = df['Y1'].apply(lambda x : int(x * height / 1000))
    df['X2'] = df['X2'].apply(lambda x : int(x * width / 1000))
    df['Y2'] = df['Y2'].apply(lambda x : int(x * height / 1000))
    labels = labels.union(set(df['label'].tolist()))
    df.to_csv(outpath + file, index=False, sep=" ")

In [29]:
###All Labels
print("Unique labels are:\n", labels)

Unique labels are:
 {'list', 'figure', 'reference', 'title', 'abstract', 'date', 'author', 'equation', 'paragraph', 'footer', 'table', 'caption', 'section'}


### Remove Unwanted Labels

In [30]:
unwanted_labels = ['table', 'equation', 'figure']

imgpath = './../../data/ocr/docbank/images/'
txtpath = './../../data/ocr/docbank/txt/'
outpath = './../../data/ocr/docbank/txt_processed/'
outimgs = './../../data/ocr/docbank/images_processed/'
gtimgs  = './../../data/ocr/docbank/gt_images/'

In [31]:
for file in os.listdir(txtpath):
    df = pd.read_csv(txtpath + file, delimiter=' ')
    img = cv2.imread(imgpath + file[:-4] + ".jpg")
    for label in unwanted_labels:
        df1 = df[df['label']==label].reset_index(drop=True)
        df2 = df[df['label']!=label].reset_index(drop=True)
        df = df2
        height, width, _ = img.shape
        for i in range(df1.shape[0]):
            X1, Y1, X2, Y2  = (df1['X1'][i], df1['Y1'][i], df1['X2'][i], df1['Y2'][i])
            df2.drop(df2[(df2['X1'] >= X1) & (df2['X2'] <= X2) & (df2['Y1'] >= Y1) & (df2['Y2'] <= Y2)].index, inplace = True)
            df2 = df2.reset_index(drop=True)
            cv2.rectangle(img, (X1, Y1), (X2, Y2), (255, 255, 255), cv2.FILLED)

        cv2.imwrite(outimgs + file[:-4] + ".jpg", img)
        df.to_csv(outpath + file, index=False, sep=" ")

### Image Ground Truths

In [32]:
for file in os.listdir(outpath):
    df = pd.read_csv(outpath + file, delimiter=' ')
    img = cv2.imread(outimgs + file[:-4] + ".jpg")
    for i in range(df.shape[0]):
        X1, Y1, X2, Y2  = (df['X1'][i], df['Y1'][i], df['X2'][i], df['Y2'][i])
        cv2.rectangle(img, (X1, Y1), (X2, Y2), (0, 255, 0), 2)

    cv2.imwrite(gtimgs + file[:-4]+'.jpg', img)